In [220]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns' , None)
pd.options.mode.copy_on_write = True

In [221]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [222]:
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/datasets/survey_results_public.csv"
DATA_PATH_PKL = "/content/drive/MyDrive/Colab Notebooks/datasets/survey_results_public.pkl"

In [223]:
raw_df = pd.read_csv(DATA_PATH)
df= raw_df.copy()

In [224]:
df.shape

(65437, 114)

In [225]:
df.head()

ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                                NaN   

                                             EdLevel  \
0                          Primary/elementary school   
1       Bachelor’s degree (B.A., B.S., B.Eng., etc.)   
2    Master’s degree (M.A., M.S., M.Eng., MBA, etc.)   
3  Some college/university study without earning ...   
4  Secondary school (e.g. American high school, G...   

                                           LearnCode  \
0                             Books / Physical media   
1  Books / Physical media;Colleague;On the job tr...   
2  Books / Physical media;Colleague;On the job tr...   
3  Other online resources (e.g., videos, blogs, f...   
4  Other online resources (e.g., videos, blogs, f...   

                                     LearnCodeOnline  \
0                                                NaN   
1  Technical documentation;Blogs;Books;Written Tu...   
2  Technical documentation;Blogs;Books;Written Tu...   
3  Stack Overflow;How-to videos;Interactive tutorial   
4  Technical documentation;Blogs;Written Tutorial...   

                                             TechDoc YearsCode YearsCodePro  \
0                                                NaN       NaN          NaN   
1  API document(s) and/or SDK document(s);User gu...        20           17   
2  API document(s) and/or SDK document(s);User gu...        37           27   
3                                                NaN         4          NaN   
4  API document(s) and/or SDK document(s);User gu...         9          NaN   

                 DevType OrgSize PurchaseInfluence BuyNewTool BuildvsBuy  \
0                    NaN     NaN               NaN        NaN        NaN   
1  Developer, full-stack     NaN               NaN        NaN        NaN   
2   Developer Experience     NaN               NaN        NaN        NaN   
3  Developer, full-stack     NaN               NaN        NaN        NaN   
4  Developer, full-stack     NaN               NaN        NaN        NaN   

  TechEndorse                                            Country Currency  \
0         NaN                           United States of America      NaN   
1         NaN  United Kingdom of Great Britain and Northern I...      NaN   
2         NaN  United Kingdom of Great Britain and Northern I...      NaN   
3         NaN                                             Canada      NaN   
4         NaN                                             Norway      NaN   

   CompTotal                             LanguageHaveWorkedWith  \
0        NaN                                                NaN   
1        NaN  Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...   
2        NaN                                                 C#   
3        NaN  C;C++;HTML/CSS;Java;JavaScript;PHP;PowerShell;...   
4        NaN            C++;HTML/CSS;JavaScript;Lua;Python;Rust   

                              LanguageWantToWorkWith  \
0                                                NaN   
1  Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...   
2  

# Functions

In [226]:
def is_splittable(pd_series,delimeter=';'):
  """Check if the column should be splitted or not"""
  return pd_series.str.contains(delimeter)


In [227]:
def split_answer(pd_series,delimeter=';'):
  """Split the answers inside the column"""
  return pd_series.str.split(delimeter)

In [228]:
def null_to_list(pd_series):
  """ Replace null value to and empty list """
  mask_null = pd_series.isnull()
  pd_series.loc[mask_null] = pd_series.loc[mask_null].apply(lambda x: [])
  return pd_series

# Processing

In [229]:
df_1 = df.copy()

In [230]:
df_1 = df.dropna(subset=['DevType'] , axis=0).reset_index(drop=True)

In [231]:
df = df_1.copy()

In [232]:
# spilt the columns which have the delmiters
object_columns = df.select_dtypes('object').columns.tolist()
for col in object_columns:

  if is_splittable(df[col]).any():
    df[col] = split_answer(df[col])
    df[col] = null_to_list(df[col])

In [233]:
replace_dict = {
    'YearsCode' : {'Less than 1 year' : 0, 'More than 50 years' : 51},
    'YearsCodePro' : {'Less than 1 year' : 0, 'More than 50 years' : 51}
}


In [234]:
# Change the columns which have diffrent datatype

for col, replacment in replace_dict.items():
  df[col] = df[col].replace(replacment).astype(np.float32)

# Visually verify results

In [240]:
i = df.sample(1).index[0]

print(df_1['LanguageHaveWorkedWith'].iloc[i])
print(df['LanguageHaveWorkedWith'].iloc[i])

i = df.sample(1).index[0]
print(df_1['DevType'].iloc[i])
print(df['DevType'].iloc[i])

i = df.sample(1).index[0]
print(df_1['YearsCodePro'].iloc[i])
print(df['YearsCodePro'].iloc[i])

HTML/CSS;Java;JavaScript;SQL
['HTML/CSS', 'Java', 'JavaScript', 'SQL']
Data engineer
Data engineer
18
18.0


In [236]:
df.to_pickle(DATA_PATH_PKL)